# DBSCAN Clustering Search

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from armscan_env.clustering import (
    TissueClusters,
    TissueLabel,
    find_DBSCAN_clusters,
)
from armscan_env.envs.rewards import anatomy_based_rwd
from armscan_env.util.visualizations import show_clusters

from config import get_config

config = get_config()

In [ ]:
path_to_mri = config.get_labels_path(1)
mri_1_label = sitk.ReadImage(path_to_mri)
mri_1_label_data = sitk.GetArrayFromImage(mri_1_label)
print(f"{mri_1_label_data.shape=}")

In [ ]:
tissues = {
    "bones": 1,
    "tendons": 2,
    "ulnar": 3,
}

DBSCAN might be a better clustering technique that offers more flexibility. It works by defining clusters as continuous regions of high density. It groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away). The DBSCAN algorithm has two parameters: `min_samples` and `eps`. The `min_samples` parameter specifies the minimum number of points that a cluster must have in order to be considered a cluster. The `eps` parameter specifies the maximum distance between two samples for one to be considered as in the neighborhood of the other.

In [ ]:
clusters_680 = TissueClusters(
    bones=find_DBSCAN_clusters(
        TissueLabel.BONES,
        mri_1_label_data[:, 680, :],
        eps=4.1,
        min_samples=46,
    ),
    tendons=find_DBSCAN_clusters(
        TissueLabel.TENDONS,
        mri_1_label_data[:, 680, :],
        eps=3.0,
        min_samples=18,
    ),
    ulnar=find_DBSCAN_clusters(
        TissueLabel.ULNAR,
        mri_1_label_data[:, 680, :],
        eps=2.0,
        min_samples=10,
    ),
)

show_clusters(clusters_680, mri_1_label_data[:, 680, :])
plt.show()

In [ ]:
sweep_loss = []
clusters_list = []

for i in range(mri_1_label_data.shape[1]):
    clusters = TissueClusters(
        bones=find_DBSCAN_clusters(
            TissueLabel.BONES,
            mri_1_label_data[:, i, :],
            eps=4.1,
            min_samples=46,
        ),
        tendons=find_DBSCAN_clusters(
            TissueLabel.TENDONS,
            mri_1_label_data[:, i, :],
            eps=3.0,
            min_samples=18,
        ),
        ulnar=find_DBSCAN_clusters(
            TissueLabel.ULNAR,
            mri_1_label_data[:, i, :],
            eps=2.0,
            min_samples=10,
        ),
    )
    loss = anatomy_based_rwd(clusters, n_landmarks=[7, 3, 1])
    clusters_list.append(clusters)
    print(f"Loss for slice {i}: {loss}")
    sweep_loss.append(loss)

In [ ]:
# Plot the loss results over the length of the data
plt.plot(range(len(sweep_loss)), sweep_loss, marker="o")

# Add labels and title
plt.xlabel("Data Length")
plt.ylabel("Loss")
plt.title("Loss Function Results")

# Show the plot
plt.show()

In [ ]:
# Find the max reward and show the slice for it
max_loss_idx = np.argmax(sweep_loss)
print(f"Minimum loss: {sweep_loss[max_loss_idx]} at index {max_loss_idx}")
fig, ax = plt.subplots(1, 2, figsize=(14, 7))
show_clusters(clusters_list[max_loss_idx], mri_1_label_data[:, max_loss_idx, :], aspect=6, ax=ax[0])
ax[1].imshow(mri_1_label_data[:, max_loss_idx, :], aspect=6, origin="lower")
plt.show()

In general, this algorithm offers a better anatomical description, since it allows to reason about the average dimension of the clusters for each kind of tissue, removing possible outliers given by segmentation errors.

In [ ]:
zero_loss_indices = np.where(np.array(sweep_loss) == 0)[0]
print(f"{len(zero_loss_indices)} indices return a zero loss: ", zero_loss_indices)

fig, axes = plt.subplots(2, 4, figsize=(21, 7))
axes = axes.flatten()
for i, idx in enumerate(zero_loss_indices):
    axes[i] = show_clusters(
        tissue_clusters=clusters_list[idx],
        slice=mri_1_label_data[:, idx, :],
        aspect=6,
        ax=axes[i],
    )
    axes[i].set_title(f"Index: {idx}, Loss: {sweep_loss[idx]:.2f}")

plt.show()

You can play around with the parameter of DBSCAN to find the best tuning, or you can use the constructor `TissueClusters.from_labelmap_slice` which iterates through the tissues to find clusters with predetermined parameters `eps` and `min_samples`.

In [ ]:
sweep_loss = []
zero_loss_clusters = []

for i in range(mri_1_label_data.shape[1]):
    clusters = TissueClusters.from_labelmap_slice(mri_1_label_data[:, i, :])
    loss = anatomy_based_rwd(clusters, n_landmarks=[7, 2, 1])
    if loss == 0:
        zero_loss_clusters.append(clusters)
    print(f"Loss for slice {i}: {loss}")
    sweep_loss.append(loss)

In [ ]:
plt.plot(range(len(sweep_loss)), sweep_loss, marker="o")
plt.xlabel("Data Length")
plt.ylabel("Loss")
plt.title("Loss Function Results")

plt.show()

In [ ]:
# Find the max reward and show the slice for it
max_loss_idx = np.argmax(sweep_loss)
print(f"Minimum loss: {sweep_loss[max_loss_idx]} at index {max_loss_idx}")
fig, ax = plt.subplots(1, 2, figsize=(14, 7))
show_clusters(clusters_list[max_loss_idx], mri_1_label_data[:, max_loss_idx, :], aspect=6, ax=ax[0])
ax[1].imshow(mri_1_label_data[:, max_loss_idx, :], aspect=6, origin="lower")
plt.show()

In [ ]:
zero_loss_indices = np.where(np.array(sweep_loss) == 0)[0]
print(f"{len(zero_loss_indices)} indices return a zero loss: ", zero_loss_indices)

fig, axes = plt.subplots(2, 4, figsize=(21, 7))
axes = axes.flatten()
for i, idx in enumerate(zero_loss_indices):
    axes[i] = show_clusters(
        tissue_clusters=zero_loss_clusters[i],
        slice=mri_1_label_data[:, idx, :],
        aspect=6,
        ax=axes[i],
    )
    axes[i].set_title(f"Index: {idx}, Loss: {sweep_loss[idx]:.2f}")

plt.show()